In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
test_set = pd.read_csv("./datasets/titanic/test.csv")
train_set = pd.read_csv("./datasets/titanic/train.csv")

In [3]:
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
from sklearn.base import TransformerMixin , BaseEstimator

class DataFrameSelector(TransformerMixin , BaseEstimator):
    def __init__(self,attributes_names):
        self.attributes_names = attributes_names
    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        return X[self.attributes_names]
    

# Inspired from stackoverflow.com/questions/25239958
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)
        

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

num_pipeline=Pipeline([
    ('num',DataFrameSelector(["Age","SibSp","Parch","Fare"])),
    ('imputer',SimpleImputer(strategy="mean"))
])

cat_pipeline=Pipeline([
    ('cat',DataFrameSelector(["Pclass","Sex","Embarked"])),
    ('cat_imputer',MostFrequentImputer()),
    ('encoder',OneHotEncoder(sparse=False))
    ])

In [7]:
from sklearn.pipeline import FeatureUnion
full_pipeline=FeatureUnion(transformer_list=[
    ('num',num_pipeline),
    ('cat',cat_pipeline)
])


In [8]:
train_set_prepared = full_pipeline.fit_transform(train_set)

In [9]:
test_set_prepared = full_pipeline.fit_transform(test_set)

In [10]:
y_train = train_set["Survived"]
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
# from sklearn.ensemble import RandomForestClassifier

# forest_clf = RandomForestClassifier(n_estimators=200,random_state=42)
# forest_clf.fit(train_set_prepared,y_train)

In [12]:
# from xgboost import XGBClassifier

# xgbooss_clf = XGBClassifier()
# xgbooss_clf.fit(train_set_prepared,y_train)

In [13]:
def save_titanic_pred(y,test_set):
    prediction = pd.DataFrame(y,columns=["Survived"],index=list(test_set["PassengerId"]))
    prediction.index.names=['PassengerId']
    prediction.to_csv("_prediction1.csv")
    return prediction

In [14]:
# y_test = forest_clf.predict(test_set_prepared)

In [15]:
# y_test_xgboost = xgbooss_clf.predict(test_set_prepared)

In [16]:
# save_titanic_pred(y_test,test_set)

In [17]:
# save_titanic_pred(y_test_xgboost,test_set)


# Voting Classifier

In [18]:
# from sklearn.ensemble import VotingClassifier
# from sklearn.svm import SVC

# forest_clf_vtg=RandomForestClassifier(n_estimators=100, random_state=0,criterion='entropy')
# xgboost_clf_vtg=XGBClassifiaer()
# svc_clf = SVC(probability=True)

# voting_clf= VotingClassifier(
#     estimators=[('fc',forest_clf_vtg),('xg', xgboost_clf_vtg),('svc',svc_clf)],
#     voting='soft')
# voting_clf.fit(train_set_prepared,y_train)



In [19]:
# y_vtg_pred = voting_clf.predict(test_set_prepared) 
# save_titanic_pred(y_vtg_pred , test_set)

# Neural Network

In [20]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
train_set_prepared_std1 = std.fit_transform(train_set_prepared)
train_set_prepared_std1

array([[-5.92480600e-01,  4.32793366e-01, -4.73673609e-01, ...,
        -4.82042680e-01, -3.07562343e-01,  6.15838425e-01],
       [ 6.38789012e-01,  4.32793366e-01, -4.73673609e-01, ...,
         2.07450510e+00, -3.07562343e-01, -1.62380254e+00],
       [-2.84663197e-01, -4.74545196e-01, -4.73673609e-01, ...,
        -4.82042680e-01, -3.07562343e-01,  6.15838425e-01],
       ...,
       [ 4.37434839e-15,  4.32793366e-01,  2.00893337e+00, ...,
        -4.82042680e-01, -3.07562343e-01,  6.15838425e-01],
       [-2.84663197e-01, -4.74545196e-01, -4.73673609e-01, ...,
         2.07450510e+00, -3.07562343e-01, -1.62380254e+00],
       [ 1.77062908e-01, -4.74545196e-01, -4.73673609e-01, ...,
        -4.82042680e-01,  3.25137334e+00, -1.62380254e+00]])

In [21]:
import tensorflow as tf

In [22]:
# X_valid = train_set_prepared_std1[750:891]
# y_valid = y_train[750:891]
# train_set_prepared_std1 = train_set_prepared_std1[0:749]
# y_train = y_train[0:749]

In [23]:
ann = tf.keras.models.Sequential()

In [24]:
ann.add(tf.keras.layers.Dense(units=300,activation="relu"))
ann.add(tf.keras.layers.Dense(units=300,activation="relu"))
ann.add(tf.keras.layers.Dense(units=300,activation="relu"))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [29]:
optimizer = tf.keras.optimizers.SGD(lr=0.5)
ann.compile(optimizer =optimizer, loss = 'binary_crossentropy' ,metrics = ['accuracy'])

In [30]:
ann.fit(train_set_prepared_std1, y_train, batch_size = 20, epochs = 500)

Epoch 1/500
45/45 [==============================] - 0s 2ms/step - loss: 0.4858 - accuracy: 0.7845
Epoch 2/500
45/45 [==============================] - 0s 2ms/step - loss: 0.4383 - accuracy: 0.8103
Epoch 3/500
45/45 [==============================] - 0s 2ms/step - loss: 0.4202 - accuracy: 0.8193
Epoch 4/500
45/45 [==============================] - 0s 3ms/step - loss: 0.4094 - accuracy: 0.8294
Epoch 5/500
45/45 [==============================] - 0s 3ms/step - loss: 0.4137 - accuracy: 0.8260
Epoch 6/500
45/45 [==============================] - 0s 2ms/step - loss: 0.4055 - accuracy: 0.8350
Epoch 7/500
45/45 [==============================] - 0s 2ms/step - loss: 0.3906 - accuracy: 0.8373
Epoch 8/500
45/45 [==============================] - 0s 3ms/step - loss: 0.4042 - accuracy: 0.8316
Epoch 9/500
45/45 [==============================] - 0s 2ms/step - loss: 0.3946 - accuracy: 0.8339
Epoch 10/500
45/45 [==============================] - 0s 2ms/step - loss: 0.3969 - accuracy: 0.8328
Epoch 11/

45/45 [==============================] - 0s 2ms/step - loss: 0.2844 - accuracy: 0.8866
Epoch 84/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2849 - accuracy: 0.8799
Epoch 85/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2939 - accuracy: 0.8810
Epoch 86/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2947 - accuracy: 0.8799
Epoch 87/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2912 - accuracy: 0.8844
Epoch 88/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2984 - accuracy: 0.8664
Epoch 89/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2905 - accuracy: 0.8788
Epoch 90/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2840 - accuracy: 0.8709
Epoch 91/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2849 - accuracy: 0.8799
Epoch 92/500
45/45 [==============================] - 0s 2ms/step - loss: 0.3171 - accuracy: 0.8642
Epoch 93/500


45/45 [==============================] - 0s 2ms/step - loss: 0.2566 - accuracy: 0.8822
Epoch 165/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2579 - accuracy: 0.8844
Epoch 166/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2634 - accuracy: 0.8956
Epoch 167/500
45/45 [==============================] - 0s 2ms/step - loss: 0.3094 - accuracy: 0.8855
Epoch 168/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2640 - accuracy: 0.8878
Epoch 169/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2507 - accuracy: 0.8866
Epoch 170/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2574 - accuracy: 0.8900
Epoch 171/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2545 - accuracy: 0.8799
Epoch 172/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2602 - accuracy: 0.8911
Epoch 173/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2535 - accuracy: 0.8889
Epoc

45/45 [==============================] - 0s 3ms/step - loss: 0.2273 - accuracy: 0.9001
Epoch 325/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2135 - accuracy: 0.9057
Epoch 326/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2326 - accuracy: 0.8956
Epoch 327/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2336 - accuracy: 0.8923
Epoch 328/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2242 - accuracy: 0.9091
Epoch 329/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2304 - accuracy: 0.9035
Epoch 330/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2150 - accuracy: 0.9057
Epoch 331/500
45/45 [==============================] - 0s 4ms/step - loss: 0.2341 - accuracy: 0.8945
Epoch 332/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2365 - accuracy: 0.8979
Epoch 333/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2182 - accuracy: 0.9001
Epoc

45/45 [==============================] - 0s 2ms/step - loss: 0.2275 - accuracy: 0.9012
Epoch 486/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2931 - accuracy: 0.8956
Epoch 487/500
45/45 [==============================] - 0s 2ms/step - loss: 0.3047 - accuracy: 0.8911
Epoch 488/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2903 - accuracy: 0.8934
Epoch 489/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2423 - accuracy: 0.8934
Epoch 490/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2239 - accuracy: 0.9068
Epoch 491/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2280 - accuracy: 0.9046
Epoch 492/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2345 - accuracy: 0.9136
Epoch 493/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2073 - accuracy: 0.9136
Epoch 494/500
45/45 [==============================] - 0s 2ms/step - loss: 0.2099 - accuracy: 0.9125
Epoc

In [ ]:
def boolstr_to_floatstr(v):
    if v == 'True':
        return '1'
    elif v == 'False':
        return '0'
    else:
        return v

In [ ]:
ann_predict1 = ann.predict(std.transform(test_set_prepared))

In [ ]:
ann_predict1 = (ann_predict1>0.5)
new_data = np.vectorize(boolstr_to_floatstr)(ann_predict1).astype(int)

In [ ]:
new_data

In [ ]:
save_titanic_pred(new_data,test_set)